In [34]:
# imports
from urllib.request import urlopen, HTTPError
from bs4 import BeautifulSoup
import re

# sentence parsing
import spacy

from random import sample, choices, seed
seed(1)

from collections import Counter
from pprint import pprint

import os

import pandas as pd

# Exercise 1: getting `n` sentences per author for `k` authors

## Constants

In [33]:
# program default parameters
AUTHOR_NUMBER = 10  # k
SENTENCE_PER_AUTHOR = 50  # n

# book selection criterion
LANGUAGE_ROLE_REGEX = re.compile(r'\((\w+)\) \(as (\w+)\)')
# - indivdual role in the book
ROLES = {'Illustrator', 'Editor', 'Compiler', 'Translator', 'Contributer', 'Dubious author', 'Author'}
ROLE = 'Author'
assert ROLE in ROLES
# - language
SCIPY_LANGUAGES = {
    'German': 'de',
    'Greek': 'el',
    'English': 'en',
    'Spanish': 'es',
    'French': 'fr',
    'Italian': 'it',
    'Dutch': 'nl',
    'Portuguese': 'pt'}
LANGUAGE = 'English'
assert LANGUAGE in SCIPY_LANGUAGES.keys()

# minimum number of books per author
BOOK_THRESHOLD = 4

# url for letter search
GUTEMBERG_LETTER_URL = 'http://www.gutenberg.org/browse/authors/{}'

# url for book download
GUTEMBERG_LETTER_UTF = "http://www.gutenberg.org{}.txt.utf-8"

VERBOSE = False

LETTERS = 'abcdefghijklmnopqrstuvwxyz'

# data save directory
SAVE_PATH = 'data'
if not os.path.isdir(SAVE_PATH):
    os.mkdir(SAVE_PATH)
    
# CSV book/bookfile name/author matchup file
CSV_FILE_PATH = os.path.join(SAVE_PATH, 'book_catalogue.csv')
CSV_HEADERS = ['author', 'book_url', 'book_title', 'book_file']

In [3]:
# scipy parser singleton
PARSERS = dict()
def get_parser(language=LANGUAGE):
    # load the sentence parser for the target language
    if language not in PARSERS.keys():
        PARSERS[language] = spacy.load(SCIPY_LANGUAGES[language])
    return PARSERS[language]


## Code

In [4]:
def get_book(book_link, base_url=GUTEMBERG_LETTER_UTF, verbose=VERBOSE):
    url = base_url.format(book_link)
    
    if verbose: print("Downloading book {} from {}".format(book_link, url))
    
    # fetching file content
    text = urlopen(url).read()
    return text

In [5]:
# TODO add check for duplicate book
def parse_page(soup: BeautifulSoup, language, role, base_book_url=GUTEMBERG_LETTER_UTF, verbose=VERBOSE):
    """
    This method contains exclusion process for books unopenable using the standard opeing method:
    - if the book's utf-8 file is not available at expected address;
    - if the book's utf-8 file is not correctly utf-8 encoded.
    
    
    """
    content_div = soup.find('div', class_='pgdbbyauthor')
    if not content_div: # if the main div is not found, return empy list of books
        yield None, []
    
    else:
        authors_lists = content_div.find_all(["ul", "h2"])
        
        for elem in authors_lists:
            if elem.name == 'h2':
                # get the first a tag with name attribute as the author name if there is at least one
                name_list = [a_tag.get_text() for a_tag in elem.find_all('a') if a_tag.has_attr('name')]
                author_name = None if len(name_list) < 1 else name_list[0]
                
            elif elem.name == 'ul' and author_name:
                book_li_list = elem.find_all('li', class_="pgdbetext")
                books = dict()
                for book_li in book_li_list:
                    # we try to extract the language and role for the book
                    match = LANGUAGE_ROLE_REGEX.search(book_li.get_text())
                    
                    if match:
                        book_language, book_role = match.group(1, 2)

                        if book_language.lower() == language.lower() and book_role.lower() == role.lower():
                            try: # try to open and decode the book, if it fails we ignore the book
                                book_title = book_li.find('a').get_text()
                                book_link = book_li.find('a')['href']
                                
                                get_book(book_link, base_url=base_book_url, verbose=verbose).decode('utf-8')
                                
                                # add the book to the list of books
                                books[book_title] = book_link
                                
                            except (UnicodeDecodeError, HTTPError):
                                if verbose: print("Book {} is either badly encoded or unavailable at this address, skipping it".format(book_link))
                                pass
                
                yield author_name, books
                
                # clean the memory to avoid getting multiple ul per author
                author_name = None

In [6]:
# TODO add check for duplicate book
def get_authors_books(letter,
                      book_anti_diplicate_set=set(),
                      language=LANGUAGE,
                      role=ROLE,
                      book_threshold=BOOK_THRESHOLD,
                      author_number=AUTHOR_NUMBER,
                      base_letter_url=GUTEMBERG_LETTER_URL,
                      base_book_url=GUTEMBERG_LETTER_UTF,
                      verbose=VERBOSE):
    url = base_letter_url.format(letter.lower())
    
    # get page
    html = urlopen(url)
    soup = BeautifulSoup(html)
    
    # extract autors and books from list
    author_dict = dict()
    for author_name, books in parse_page(soup, language, role, base_book_url=base_book_url, verbose=verbose):
        books = {book_link: book_title for book_link, book_title in books.items()
                 if book_link not in book_anti_diplicate_set} # remove books already selected
        if len(books) >= book_threshold:
            # add author and books to the list if there is enough books
            author_dict[author_name] = books
            book_anti_diplicate_set |= books.keys()
            
            if verbose: print(author_name, books)
            
        # if we have reached the number of authors we want, break the loop
        if len(author_dict) >= author_number: break
            
    return author_dict, book_anti_diplicate_set

In [42]:
# pick sentences from a book
def get_book_sents(book_link, sentence_number, language=LANGUAGE, base_url=GUTEMBERG_LETTER_UTF, verbose=VERBOSE):
    # get the content of the book
    book_text = get_book(book_link, base_url, verbose=verbose).decode('utf-8')
    book_text = book_text.split("***",2)[-1].strip() # removing gutenberg header
    
    # sentence tokenize using spacy
    book_text = book_text[:500000]  # taking only a sample of the book as spacy puts a limit on the number of characters
    if verbose: print("Parsing book {} with spacy".format(book_link))
    parsed_text = get_parser(language)(book_text)
    sentences = [sent.text for sent in parsed_text.sents]
    
    # random sampling of sentences
    chosen_sentences = sample(sentences, k=sentence_number)
    
    # removing extra spacing characters from selected lines
    chosen_sentences = [re.sub(r'\s+', ' ', sent).strip() for sent in chosen_sentences]
    return chosen_sentences

In [46]:
def get_sentence_per_book(books, sentence_number=SENTENCE_PER_AUTHOR, strategy='random', verbose=VERBOSE):  #decide number of lines per book
    if strategy=='random':
        sentence_per_book_counter = Counter(choices(list(books.values()), k=sentence_number))
        
    elif strategy=='balanced':
        # select the number of sentences to pick from each book, equally divided accorss the books
        sentence_per_book_counter = Counter()
        while sum(sentence_per_book_counter.values()) < sentence_number:
            for book_url in books.values():
                sentence_per_book_counter[book_url] += 1
                if sum(sentence_per_book_counter.values()) >= sentence_number:
                    break
                
    if verbose:
        print("Sentence repartition:")
        pprint(sentence_per_book_counter)
        
    return sentence_per_book_counter

In [19]:
def get_sentences(author_dict,
                  sentence_number=SENTENCE_PER_AUTHOR,
                  language=LANGUAGE,
                  base_book_url=GUTEMBERG_LETTER_UTF,
                  verbose=VERBOSE):
    sentence_dict = dict()
    for author, books in author_dict.items():
        # select the number of sentences to pick from each book of the author
        line_per_book_counter = get_sentence_per_book(books, sentence_number, verbose=verbose)
        
        # load the sentences from each book
        sentence_dict[author] = dict()
        for book_url, book_sentence_number in line_per_book_counter.items():
            sentences = get_book_sents(book_url,
                                       book_sentence_number,
                                       base_url=base_book_url,
                                       language=language,
                                       verbose=verbose)
            sentence_dict[author][book_url] = sentences
            if verbose: 
                print("Sentences from {}:".format(book_url))
                pprint(sentences)
    
    return sentence_dict

In [10]:
def author_per_letter(author_number=AUTHOR_NUMBER, letters=LETTERS):
    """Return a Counter object over randomly chosen letters among 'letters'.
    The total of the elements in the counter is equal to 'author_number'.
    
    :param author_number: number of elements to choose accross the letters (int)
    :param letters: sequence of elements to choose from (itterable)
    
    :return: a counter where keys are letters and values add up to author_number (Counter)"""
    return Counter(choices(letters, k=author_number))

In [48]:
def get_file_name(book_url, path=SAVE_PATH):
    return os.path.join(path, book_url.split('/')[-1] + ".txt")

def save(sentence_dict,
         author_dict,
         book_path=SAVE_PATH,
         csv_path=CSV_FILE_PATH,
         csv_headers=CSV_HEADERS,
         verbose=VERBOSE):
    book_to_path = dict()
    for author_sentence_dict in sentence_dict.values():
        for book_url, book_sentences in author_sentence_dict.items():
            file_name = get_file_name(book_url, path=book_path)

            with open(file_name, 'w') as f:
                f.write("\n".join(book_sentences))

                # stores the path to the file
                book_to_path[book_url] = file_name
                if verbose: print("{} line(s) from book '{}' written to '{}'".format(len(book_sentences), book_url, file_name))
    
    # save the information about books from which sentences were chosen (their URL, their title, and the file where the sentences were saved)
    csv_data = [[author_name, book_url, book_title, book_to_path[book_url]]
                for author_name, books in author_dict.items()
                for book_title, book_url in books.items()
                if book_url in book_to_path.keys()]
    
    # write the data to CSV
    df = pd.DataFrame(csv_data, columns=csv_headers)
    df.to_csv(CSV_FILE_PATH)
    if verbose: print("book catalogue written to {}".format(csv_path))

In [49]:
def exercice_1(author_number=AUTHOR_NUMBER,
               sentence_number=SENTENCE_PER_AUTHOR,
               language=LANGUAGE,
               role=ROLE,
               book_threshold=BOOK_THRESHOLD,
               base_letter_url=GUTEMBERG_LETTER_URL,
               base_book_url=GUTEMBERG_LETTER_UTF,
               letters=LETTERS,
               book_path=SAVE_PATH,
               csv_path=CSV_FILE_PATH,
               csv_headers=CSV_HEADERS,
               verbose=VERBOSE):
    book_anti_diplicate_set = set()
    author_dict = dict()
    
    # Step 1: selecting the k authors
    for letter, count in author_per_letter(author_number=author_number, letters=letters).items():
        if verbose: print("Picking {} author(s) from letter {}".format(count, letter))
            
        # generate a dictionnary of authors and their books
        author_dict_temp, book_anti_diplicate_set = get_authors_books(letter,
                                                                      book_anti_diplicate_set=book_anti_diplicate_set, 
                                                                      language=language,
                                                                      book_threshold=book_threshold,
                                                                      author_number=count,
                                                                      base_letter_url=base_letter_url,
                                                                      base_book_url=base_book_url,
                                                                      verbose=verbose)
        author_dict.update(author_dict_temp)
        
    # Step 2: getting n sentences per author
    sentence_dict = get_sentences(author_dict,
                                  sentence_number=sentence_number,
                                  language=language,
                                  base_book_url=base_book_url,
                                  verbose=verbose)
    
    # Step 3: storing the sentences, 1 file per book, adn 1 file to link the books to their main author
    save(sentence_dict,
         author_dict,
         book_path=book_path,
         csv_path=csv_path,
         csv_headers=csv_headers,
         verbose=verbose)
    return author_dict, sentence_dict
        
print(exercice_1(verbose=True))

Picking 2 author(s) from letter t
Book is either badly encoded or unavailable at this address: /ebooks/15017
Tacitus, Cornelius, 56-117 {'Arguments of Celsus, Porphyry, and the Emperor Julian, Against the Christians\rAlso Extracts from Diodorus Siculus, Josephus, and Tacitus, Relating to the Jews, Together with an Appendix': '/ebooks/37696', 'Germania and Agricola': '/ebooks/9090', 'The Germany and the Agricola of Tacitus': '/ebooks/7524', 'The Reign of Tiberius, Out of the First Six Annals of Tacitus;\rWith His Account of Germany, and Life of Agricola': '/ebooks/7959', 'Tacitus on Germany': '/ebooks/2995', 'Tacitus: The Histories, Volumes I and II': '/ebooks/16927'}
Book is either badly encoded or unavailable at this address: /ebooks/49804
Book is either badly encoded or unavailable at this address: /ebooks/10208
Book is either badly encoded or unavailable at this address: /ebooks/56823
Taggart, Marion Ames, 1866-1945 {'The Blissylvania Post-Office': '/ebooks/48552', 'The Daughters of

Book is either badly encoded or unavailable at this address: /ebooks/55080
Habberton, John, 1842-1921 {'All He Knew: A Story': '/ebooks/14895', 'Caleb Wright: A Story of the West': '/ebooks/43994', "Helen's Babies": '/ebooks/4281', 'Romance of California Life\rIllustrated by Pacific Slope Stories, Thrilling, Pathetic and Humorous': '/ebooks/13832', "The Scripture Club of Valley Rest; or, Sketches of Everybody's Neighbours": '/ebooks/54627', 'Trif and Trixy\rA story of a dreadfully delightful little girl and her adoring and tormented parents, relations, and friends': '/ebooks/51788'}
Picking 1 author(s) from letter j
Book is either badly encoded or unavailable at this address: /ebooks/57662
Book is either badly encoded or unavailable at this address: /ebooks/30548
Book is either badly encoded or unavailable at this address: /ebooks/40401
Jackson, Gabrielle E. (Gabrielle Emilie), 1861- {'Caps and Capers: A Story of Boarding-School Life': '/ebooks/26549', 'A Dixie School Girl': '/ebooks/2

Book is either badly encoded or unavailable at this address: /ebooks/56264
Book is either badly encoded or unavailable at this address: /ebooks/57836
Book is either badly encoded or unavailable at this address: /ebooks/49795
Book is either badly encoded or unavailable at this address: /ebooks/23893
Zangwill, Israel, 1864-1926 {'The Big Bow Mystery': '/ebooks/28164', 'Children of the Ghetto: A Study of a Peculiar People': '/ebooks/12680', 'Chosen PeoplesBeing the First "Arthur Davis Memorial Lecture" delivered before the Jewish Historical Society at University College on Easter-Passover Sunday, 1918/5678': '/ebooks/20631', 'Ghetto Comedies': '/ebooks/28982', 'Ghetto Tragedies': '/ebooks/35076', 'The Grandchildren of the Ghetto': '/ebooks/35238', 'The Grey Wig: Stories and Novelettes': '/ebooks/16408', 'The King of Schnorrers: Grotesques and Fantasies': '/ebooks/38413', 'Merely Mary Ann': '/ebooks/20078', "The Old Maids' Club": '/ebooks/45623', 'Without Prejudice': '/ebooks/6304'}
Pickin

 'they render more full and sonorous by applying their mouths to their '
 'shields.',
 'Thus Pliny, xv.']
Parsing book /ebooks/9090 with spacy
Sentences from /ebooks/9090:
['Epis.',
 'But the Germans, though they lived so much on milk, did not understand the '
 'art of making cheese, see Pliny, N.H. 11, 96. "',
 'Rush forth to meet, penetrantibus_, etc.',
 'Juxta libertatem_',
 '_of the stronger party',
 '-ere_. Cf.',
 'Section 2.',
 'Very many of the Chatti are']
Parsing book /ebooks/37696 with spacy
Sentences from /ebooks/37696:
['Lastly, he asserts, that whatever is related as far as to the reign of '
 'Antiochus contains a true history; but that all that is said posterior to '
 'this time, as the writer was ignorant of futurity, is false."',
 'It is by no means therefore proper to do an injury.',
 '* i. e. beneficent dæmonss; for the ancients divided dæmonss into the '
 'beneficent and malevolent.',
 'When they have destroyed one temple they run to another, and a third, and '
 'tro

 'refusals to Aunt Henrietta and dear Mrs. Van Alyn for their offers."',
 '"To Boston, and \'it may be for years and it may be forever.\'',
 "Nothing splendid was retained; only the pictures in the girls' rooms, their "
 "own special pet chairs, desks, tables, Bab's piano, and Mr. Wyndham's "
 'library chair.',
 'I have sat over my embroidery without a breath of air for five days, and I '
 'was nearly wild.']
Parsing book /ebooks/48552 with spacy
Sentences from /ebooks/48552:
['cried Jack, rolling over in spasms of laughter, while Miss Isabel, laughing, '
 "too, at Beatrice's funny appearance and remark, helped get her up.",
 '"I want you all to make a novena for me, and begin right off to-night.',
 "And I didn't think so, but I had to go.",
 'Do not charge a fee for access to, viewing, displaying, performing, copying '
 'or distributing any Project Gutenberg-tm works unless you comply with '
 'paragraph 1.E.8 or 1.E.9.',
 'I wonder why nice people are not always happy?',
 'Dr. Gregory

...

 "He took Katharine's letter from his breast, spread it open on the coverlet, "
 'stretching his arms out round it, like a frame.',
 'Many small donations ($1 to $5,000) are particularly important to '
 'maintaining tax exempt status with the IRS.',
 'and I know you do, old nurse, I know you do--who']
Parsing book /ebooks/59207 with spacy
Sentences from /ebooks/59207:
['"I find every body\'s the same age,--seven-and-twenty.',
 "There was an odour of patchouli, too, about it which roused Beresford's ire, "
 'and he muttered as he opened it, "Confounded stuff!',
 ", I'll say good night.",
 'He may go off at any moment; his life is not certain for an hour;',
 '"What do you mean by his own set, Mr. Beresford?" said Lyster, rousing '
 'himself. "',
 'Once in the room, Mr. Pringle toned down visibly, and conducted himself like '
 'an ordinary mortal.',
 'And Alice called on Barbara, and petted her and praised her, and in her '
 'silly little way did every thing possible to prevent the smalle

 'But it is still gay with music, virtue triumphs on, and vice grovels at the '
 'penitent form.']
Parsing book /ebooks/20631 with spacy
Sentences from /ebooks/20631:
['Destroy them from under the heavens of the Lord!"',
 'General Information About Project Gutenberg-',
 'Delenda est',
 'You may convert to and distribute this work in any binary, compressed, '
 'marked up, nonproprietary or proprietary form, including any word processing '
 'or hypertext form.',
 '1.E.9.']
Parsing book /ebooks/28982 with spacy
Sentences from /ebooks/28982:
['But Kloot!',
 'And Rachel, that pretty, clinging cherub!',
 'As he lay in his cell he chewed the cud of revenge.',
 'Elias followed, chattering with feverish gaiety.']
Parsing book /ebooks/20078 with spacy
Sentences from /ebooks/20078:
['She loomed large to him in the match-light--he seemed to see her through a '
 'golden haze.']
Sentence repartition:
Counter({'/ebooks/21117': 6,
         '/ebooks/32920': 4,
         '/ebooks/32933': 4,
         '/eb

['neither could Vere herself, and I tried hard to calm myself so as not to '
 'excite her too much.',
 "I'll begin at the beginning, and tell it straight through to the end.",
 'She looks quite young, and she was the beauty of the county when she was a '
 'girl, and I never did see in all my life anybody so immaculately perfect in '
 'appearance!']
Parsing book /ebooks/21099 with spacy
Sentences from /ebooks/21099:
['Peggy _Darcy_!']
Parsing book /ebooks/32936 with spacy
Sentences from /ebooks/32936:
['Inadvertently also to meet a nice man...',
 '"Very clever." "Original?" "Original!"',
 'Friends also ceased in due time to forward notes of ostensible '
 'congratulation, behind which the real amaze was plainly stamped; only one '
 'effect was of any lasting nature, and regarding this Martin felt an odd '
 'mixture of chagrin and elation.']
Sentence repartition:
Counter({'/ebooks/10003': 15,
         '/ebooks/14029': 14,
         '/ebooks/37953': 11,
         '/ebooks/38825': 10})
Parsin

 "received him charmingly, was most kind and courteous, but didn't do what the "
 'man wanted.',
 'I had no particular ties nor traditions, had no grandfather killed on the '
 'scaffold, nor frozen to death in the retreat of "La Grande Armée" from '
 'Moscow.',
 'I had my carriage and went alone.',
 "He didn't say a word about music, either then or on a subsequent occasion "
 'when I lunched with him at the house of a great friend and admirer, who was '
 'a beautiful musician.',
 'They invaded charmed circles, walked boldly up to archdukes and duchesses, '
 'talking to them cheerfully and easily without waiting to be spoken to, '
 'giving them a great deal of information upon all subjects, Austrian as well '
 'as American, and probably interested the very stiff Austrian royalties much '
 'more than the ordinary trained diplomatist, who would naturally be more '
 'correct in his attitude and conversation.',
 'I am speaking of course of purely French marriages.',
 'If you do not charge a

({'Tacitus, Cornelius, 56-117': {'Arguments of Celsus, Porphyry, and the Emperor Julian, Against the Christians\rAlso Extracts from Diodorus Siculus, Josephus, and Tacitus, Relating to the Jews, Together with an Appendix': '/ebooks/37696', 'Germania and Agricola': '/ebooks/9090', 'The Germany and the Agricola of Tacitus': '/ebooks/7524', 'The Reign of Tiberius, Out of the First Six Annals of Tacitus;\rWith His Account of Germany, and Life of Agricola': '/ebooks/7959', 'Tacitus on Germany': '/ebooks/2995', 'Tacitus: The Histories, Volumes I and II': '/ebooks/16927'}, 'Taggart, Marion Ames, 1866-1945': {'The Blissylvania Post-Office': '/ebooks/48552', 'The Daughters of the Little Grey House': '/ebooks/48604', 'The Little Grey House': '/ebooks/48363', 'A Pilgrim Maid: A Story of Plymouth Colony in 1620': '/ebooks/39323', 'Six Girls and Bob: A Story of Patty-Pans and Green Fields': '/ebooks/47655', 'Six Girls and the Tea Room': '/ebooks/48389', 'The Wyndham Girls': '/ebooks/47739'}, 'Quick